In [1]:
from pathlib import Path
import os
import sys

sys.path.append(str(Path(os.getcwd()).parent.parent))

In [2]:
import pandas as pd
import numpy as np
import torch

In [3]:
df = pd.read_csv('../../data/Petitions.csv')[:10000]
X, y_name = list(df['public_petition_text']), list(df['reason_category'])

In [4]:
df

,id,public_petition_text,reason_category
0,3168490,снег на дороге,Благоустройство
1,3219678,очистить кабельный киоск от рекламы,Благоустройство
2,2963920,"Просим убрать все деревья и кустарники, которы...",Благоустройство
3,3374910,Неудовлетворительное состояние парадной - надп...,Содержание МКД
4,3336285,Граффити,Благоустройство
...,...,...,...
9995,3236509,очистите о о,Благоустройство
9996,3213091,"На фасаде дома, незаконно и без разрешающих до...",Нарушение правил пользования общим имуществом
9997,3242261,Товарный переулок. Мусор. В администрацию Цент...,Благоустройство
9998,3311922,В проезжей части просел канализационный люк.\n...,Повреждения или неисправность элементов улично...


In [5]:
from sklearn.preprocessing import LabelEncoder


le = LabelEncoder()
y = le.fit_transform(y_name)

In [6]:
y, le.classes_

(array([0, 0, 0, ..., 0, 8, 0], dtype=int64),
 array(['Благоустройство', 'Водоотведение', 'Водоснабжение', 'Кровля',
        'Нарушение порядка пользования общим имуществом',
        'Нарушение правил пользования общим имуществом',
        'Незаконная информационная и (или) рекламная конструкция',
        'Незаконная реализация товаров с торгового оборудования (прилавок, ящик, с земли)',
        'Повреждения или неисправность элементов уличной инфраструктуры',
        'Подвалы', 'Санитарное состояние', 'Содержание МКД',
        'Состояние рекламных или информационных конструкций', 'Фасад',
        'Центральное отопление'], dtype='<U80'))

## 0. Подготовка данных

In [7]:
from mylib.preprocessing.preprocessing import preprocessing
from nltk.corpus import stopwords
import string


sw = stopwords.words('russian')
sw.extend(list(string.punctuation))

sub_expressions = {
    '<[^>]*>': "",
    "&(.*);": "",
    "/": " ",
    "[^\s^\w^-]+": "",
    " -* ": " ",
    " \d+ ": " ",
}

dataset = list(map(lambda s: 
                   preprocessing(s, ["apply_regular_expressions", "tokenize", "lemmatization", "drop_stopwords"], 
                                 sub_expressions=sub_expressions, tokenization_type='word', stopwords=sw), list(df['public_petition_text'])))

In [151]:
dataset

[['снег', 'дорога'],
 ['очистить', 'кабельный', 'киоск', 'реклама'],
 ['просить',
  'убрать',
  'всё',
  'дерево',
  'кустарник',
  'который',
  'выйти',
  'предел',
  'газон',
  'пешеходный',
  'зона',
  'начинать',
  'подъезд',
  'подъезд',
  'фасад',
  'дом',
  'сторона',
  'ул',
  'наличный'],
 ['неудовлетворительный', 'состояние', 'парадный', 'надпись', 'дверь', 'этаж'],
 ['граффити'],
 ['необходимо',
  'проверить',
  'законность',
  'установка',
  'вывеска',
  'фасад',
  'мкд',
  'адрес',
  'проспект',
  'непокорённый',
  'случай',
  'вывеска',
  'установить',
  'незаконно',
  'её',
  'необходимо',
  'демонтировать'],
 ['уборка',
  'производиться',
  'лестница',
  'очень',
  'грязно',
  'весь',
  'этаж',
  'вплоть',
  '5-й',
  'звонок',
  'жкс2',
  'дать',
  'результат'],
 ['мусор'],
 ['отсутствовать',
  'освещение',
  'лестничный',
  'площадка',
  'этаж',
  'парадный',
  '2'],
 ['делать', 'благоустройство', 'никто', 'убирать', 'мусор', 'ежедневно'],
 ['просьба', 'закрасить'],
 [

In [8]:
# text_corpus = [word for text_document in dataset for word in text_document if len(word) > 1 and word.isalpha()]
dataset_no_digits = [[word for word in text_document if len(word) > 1 and word.isalpha()] for text_document in dataset]
dataset_no_digits = [text_document for text_document in dataset_no_digits if text_document != []]

In [9]:
vocabulary = set([word for text_document in dataset_no_digits for word in text_document])

In [152]:
vocabulary

{'леквидировать',
 'перекладывать',
 'савушкина',
 'локальный',
 'ствол',
 'катать',
 'плохо',
 'малярный',
 'опасный',
 'выборы',
 'помывочный',
 'доделать',
 'копия',
 'налево',
 'раскрыть',
 'неприглядный',
 'проживание',
 'окраска',
 'инженер',
 'двойной',
 'табак',
 'просушка',
 'гриль',
 'переставить',
 'лита',
 'миллер',
 'снять',
 'следить',
 'выставление',
 'переселенец',
 'напряжение',
 'убедиться',
 'визуально',
 'недостаточный',
 'продумать',
 'незасохнуть',
 'вопиять',
 'шина',
 'тонас',
 'контролируещий',
 'троллейбус',
 'раздаваться',
 'ожидание',
 'искривить',
 'непроходимый',
 'загородный',
 'трамвайный',
 'отсутствуетта',
 'супермаркет',
 'полуподвальный',
 'жэс',
 'частота',
 'порча',
 'обломиться',
 'автотранспортный',
 'смольный',
 'нацарапать',
 'ключий',
 'добрый',
 'дальний',
 'превышать',
 'зао',
 'скрипеть',
 'баскетбольный',
 'уметь',
 'шахтеэлектрический',
 'вандализм',
 'неверно',
 'использоваться',
 'протокол',
 'ямахпросить',
 'муфта',
 'мичманский',
 'пр

In [10]:
id_word = {id: word for id, word in enumerate(vocabulary, 1)}

In [153]:
id_word

{1: 'леквидировать',
 2: 'перекладывать',
 3: 'савушкина',
 4: 'локальный',
 5: 'ствол',
 6: 'катать',
 7: 'плохо',
 8: 'малярный',
 9: 'опасный',
 10: 'выборы',
 11: 'помывочный',
 12: 'доделать',
 13: 'копия',
 14: 'налево',
 15: 'раскрыть',
 16: 'неприглядный',
 17: 'проживание',
 18: 'окраска',
 19: 'инженер',
 20: 'двойной',
 21: 'табак',
 22: 'просушка',
 23: 'гриль',
 24: 'переставить',
 25: 'лита',
 26: 'миллер',
 27: 'снять',
 28: 'следить',
 29: 'выставление',
 30: 'переселенец',
 31: 'напряжение',
 32: 'убедиться',
 33: 'визуально',
 34: 'недостаточный',
 35: 'продумать',
 36: 'незасохнуть',
 37: 'вопиять',
 38: 'шина',
 39: 'тонас',
 40: 'контролируещий',
 41: 'троллейбус',
 42: 'раздаваться',
 43: 'ожидание',
 44: 'искривить',
 45: 'непроходимый',
 46: 'загородный',
 47: 'трамвайный',
 48: 'отсутствуетта',
 49: 'супермаркет',
 50: 'полуподвальный',
 51: 'жэс',
 52: 'частота',
 53: 'порча',
 54: 'обломиться',
 55: 'автотранспортный',
 56: 'смольный',
 57: 'нацарапать',
 58:

In [11]:
word_id = {word: id for id, word in id_word.items()}

In [154]:
word_id

{'леквидировать': 1,
 'перекладывать': 2,
 'савушкина': 3,
 'локальный': 4,
 'ствол': 5,
 'катать': 6,
 'плохо': 7,
 'малярный': 8,
 'опасный': 9,
 'выборы': 10,
 'помывочный': 11,
 'доделать': 12,
 'копия': 13,
 'налево': 14,
 'раскрыть': 15,
 'неприглядный': 16,
 'проживание': 17,
 'окраска': 18,
 'инженер': 19,
 'двойной': 20,
 'табак': 21,
 'просушка': 22,
 'гриль': 23,
 'переставить': 24,
 'лита': 25,
 'миллер': 26,
 'снять': 27,
 'следить': 28,
 'выставление': 29,
 'переселенец': 30,
 'напряжение': 31,
 'убедиться': 32,
 'визуально': 33,
 'недостаточный': 34,
 'продумать': 35,
 'незасохнуть': 36,
 'вопиять': 37,
 'шина': 38,
 'тонас': 39,
 'контролируещий': 40,
 'троллейбус': 41,
 'раздаваться': 42,
 'ожидание': 43,
 'искривить': 44,
 'непроходимый': 45,
 'загородный': 46,
 'трамвайный': 47,
 'отсутствуетта': 48,
 'супермаркет': 49,
 'полуподвальный': 50,
 'жэс': 51,
 'частота': 52,
 'порча': 53,
 'обломиться': 54,
 'автотранспортный': 55,
 'смольный': 56,
 'нацарапать': 57,
 'кл

In [12]:
dataset_id = [list(pd.DataFrame(text_document)[0].map(word_id)) for text_document in dataset_no_digits]

In [155]:
dataset_id

[[3972, 7160],
 [2449, 3822, 7995, 4976],
 [2708,
  4466,
  6999,
  6936,
  3350,
  6487,
  2648,
  3194,
  721,
  137,
  669,
  2448,
  3025,
  3025,
  4459,
  7470,
  8092,
  3158,
  8046],
 [1048, 2566, 6342, 580, 8127, 5787],
 [6392],
 [3752,
  4553,
  6160,
  5535,
  7225,
  4459,
  5316,
  1882,
  3329,
  6569,
  1642,
  7225,
  742,
  5155,
  3394,
  3752,
  2052],
 [633, 5994, 164, 5746, 4964, 1159, 5787, 5064, 1940, 2824, 1816],
 [1680],
 [3925, 3141, 1908, 7362, 5787, 6342],
 [2401, 7163, 5291, 6275, 1680, 5017],
 [6591, 2309],
 [4976, 6692],
 [3972, 885, 4466],
 [3970, 4685, 5234, 1680],
 [6885],
 [5796, 7404, 391, 7238, 1005, 6138, 721, 7470, 1882, 5005, 4792, 6875],
 [1680, 4801],
 [6283, 5717, 6342, 2857, 5364, 4248, 2708, 1569],
 [3811, 6336, 6591, 498, 1240, 5787],
 [7943, 633, 5005, 1680, 885, 721, 6522, 7467, 2282],
 [580, 6692],
 [4466],
 [3091, 3691, 6181, 2260, 7362],
 [2092,
  5654,
  7225,
  4459,
  6614,
  663,
  1448,
  4761,
  4834,
  6989,
  6150,
  3701,
  5

In [13]:
text_corpus_id = torch.nn.utils.rnn.pad_sequence([torch.tensor(text_document) for text_document in dataset_id], batch_first=True).numpy()

In [156]:
text_corpus_id

array([[3972, 7160,    0, ...,    0,    0,    0],
       [2449, 3822, 7995, ...,    0,    0,    0],
       [2708, 4466, 6999, ...,    0,    0,    0],
       ...,
       [2889, 6984, 1680, ...,    0,    0,    0],
       [3769, 8093, 1537, ...,    0,    0,    0],
       [ 580, 3565,    0, ...,    0,    0,    0]], dtype=int64)

In [14]:
from gensim.models import Word2Vec


w2v = Word2Vec(sentences=dataset_no_digits, min_count=1, vector_size=64, epochs=50)

In [15]:
id_vec = {id: w2v.wv[word] for id, word in id_word.items()}
id_vec[0] = np.zeros(len(id_vec[1]))

In [157]:
id_vec

{1: array([-7.7442288e-02, -1.5033711e-01, -2.1181734e-02,  9.5886745e-02,
        -6.2200498e-02,  9.6855313e-02,  5.1350416e-06, -3.1162071e-01,
        -3.0567020e-01, -1.9402827e-01,  9.9559076e-02,  7.3860787e-02,
        -3.1047115e-01,  7.6049000e-02,  1.1544620e-02,  2.9531407e-01,
         4.1168250e-02, -2.3204534e-01, -3.9858449e-02, -1.4812577e-03,
         1.5937166e-01,  2.5201231e-01,  2.8843540e-01, -2.1468845e-01,
        -7.9984004e-03, -1.2160952e-01, -1.4311212e-01,  1.8900035e-01,
         7.7289209e-02,  1.2247331e-01,  5.8605958e-02, -8.1238281e-03,
        -9.5322594e-02, -3.1247801e-01,  4.3193204e-03, -1.0429532e-02,
        -3.7949327e-02, -1.0742139e-01, -1.5262133e-02,  1.2748073e-01,
        -1.9854402e-02,  1.7984192e-01, -6.8241931e-02,  8.5037455e-02,
         2.4722026e-01, -2.3247851e-01,  1.1927252e-01, -1.6832557e-01,
        -1.2966620e-02, -1.7075303e-01,  1.0257215e-02,  1.5808910e-01,
        -1.6619336e-02,  7.4756645e-02,  9.3040772e-02,  1.64

In [16]:
text_corpus_vec = np.array(
    [np.array(
        [id_vec[word_id] for word_id in text_document_id]
    ) for text_document_id in text_corpus_id]
)

In [17]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(text_corpus_vec, y, test_size=0.1, stratify=y, shuffle=True)

In [51]:
TRAIN_N, SEQ_LEN, INPUT_SIZE = X_train.shape
HIDDEN_SIZE = 128
TRAIN_N, SEQ_LEN, INPUT_SIZE

(9000, 188, 64)

In [94]:
from torch.utils.data import TensorDataset, DataLoader


train_ds = TensorDataset(torch.from_numpy(X_train).type(torch.float32), torch.from_numpy(y_train.reshape(-1, 1)).type(torch.long))
train_dl = DataLoader(train_ds, batch_size=200)

## 1. RNN

In [96]:
import torch.nn as nn


class RNNModel(nn.Module):
    def __init__(self, input_size: int, hidden_size: int) -> None:
        super(RNNModel, self).__init__()
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, 15)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out, h = self.rnn(x)
        y = self.linear(h.squeeze(0))
        return y


model_rnn = RNNModel(INPUT_SIZE, HIDDEN_SIZE)
loss_rnn = nn.CrossEntropyLoss()
optimizer_rnn = torch.optim.Adam(model_rnn.parameters(), lr=0.0025)

In [158]:
sm = nn.Softmax(1)

In [99]:
epochs = 30
for epoch in range(epochs):
    for x_b, y_b in train_dl:
        output = model_rnn(x_b)
        loss_value = loss_rnn(output, y_b.squeeze(1))
        optimizer_rnn.zero_grad()
        loss_value.backward()
        optimizer_rnn.step()
        
    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')

Эпоха 1, Значение функции потерь: 1.4028106927871704
Эпоха 2, Значение функции потерь: 1.4017959833145142
Эпоха 3, Значение функции потерь: 1.4013842344284058
Эпоха 4, Значение функции потерь: 1.4017529487609863
Эпоха 5, Значение функции потерь: 1.4013508558273315
Эпоха 6, Значение функции потерь: 1.401676893234253
Эпоха 7, Значение функции потерь: 1.4010803699493408
Эпоха 8, Значение функции потерь: 1.4012696743011475
Эпоха 9, Значение функции потерь: 1.400976538658142
Эпоха 10, Значение функции потерь: 1.4006935358047485
Эпоха 11, Значение функции потерь: 1.4001662731170654
Эпоха 12, Значение функции потерь: 1.400339961051941
Эпоха 13, Значение функции потерь: 1.4001703262329102
Эпоха 14, Значение функции потерь: 1.399245023727417
Эпоха 15, Значение функции потерь: 1.4001635313034058
Эпоха 16, Значение функции потерь: 1.3998537063598633
Эпоха 17, Значение функции потерь: 1.3994030952453613
Эпоха 18, Значение функции потерь: 1.3989176750183105
Эпоха 19, Значение функции потерь: 1.3994

In [119]:
from sklearn.metrics import classification_report


y_pred_rnn = np.argmax(sm(model_rnn(torch.from_numpy(X_test).type(torch.float32))).detach().numpy(), axis=1)
print(classification_report(y_pred_rnn, y_test))

              precision    recall  f1-score   support

           0       1.00      0.58      0.74      1000
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0

    accuracy                           0.58      1000
   macro avg       0.07      0.04      0.05      1000
weighted avg       1.00   

c:\Users\bugae\nlp\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bugae\nlp\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bugae\nlp\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 2. LSTM

In [145]:
class LSTMModel(nn.Module):
    def __init__(self, input_size: int, hidden_size: int) -> None:
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.linear = nn.Linear(2 * hidden_size, 15)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out, (h, c) = self.lstm(x)
        y = self.linear(torch.hstack((h.squeeze(0), c.squeeze(0))))
        return y


model_lstm = LSTMModel(INPUT_SIZE, HIDDEN_SIZE)
loss_lstm = nn.CrossEntropyLoss()
optimizer_lstm = torch.optim.Adam(model_lstm.parameters(), lr=0.0025)

In [146]:
epochs = 20
for epoch in range(epochs):
    for x_b, y_b in train_dl:
        output = model_lstm(x_b)
        loss_value = loss_lstm(output, y_b.squeeze(1))
        optimizer_lstm.zero_grad()
        loss_value.backward()
        optimizer_lstm.step()
        
    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')

Эпоха 1, Значение функции потерь: 1.395838975906372
Эпоха 2, Значение функции потерь: 1.3978886604309082
Эпоха 3, Значение функции потерь: 1.3977550268173218
Эпоха 4, Значение функции потерь: 1.3977161645889282
Эпоха 5, Значение функции потерь: 1.3976943492889404
Эпоха 6, Значение функции потерь: 1.397635817527771
Эпоха 7, Значение функции потерь: 1.3975261449813843
Эпоха 8, Значение функции потерь: 1.3973686695098877
Эпоха 9, Значение функции потерь: 1.3971806764602661
Эпоха 10, Значение функции потерь: 1.3969820737838745
Эпоха 11, Значение функции потерь: 1.3967889547348022
Эпоха 12, Значение функции потерь: 1.396615743637085
Эпоха 13, Значение функции потерь: 1.3964706659317017
Эпоха 14, Значение функции потерь: 1.396353006362915
Эпоха 15, Значение функции потерь: 1.3962621688842773
Эпоха 16, Значение функции потерь: 1.3962020874023438
Эпоха 17, Значение функции потерь: 1.3961553573608398
Эпоха 18, Значение функции потерь: 1.396111011505127
Эпоха 19, Значение функции потерь: 1.39613

In [147]:
y_pred_lstm = np.argmax(sm(model_lstm(torch.from_numpy(X_test).type(torch.float32))).detach().numpy(), axis=1)
print(classification_report(y_pred_lstm, y_test))

              precision    recall  f1-score   support

           0       1.00      0.58      0.74      1000
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         0
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         0
           7       0.00      0.00      0.00         0
           8       0.00      0.00      0.00         0
           9       0.00      0.00      0.00         0
          10       0.00      0.00      0.00         0
          11       0.00      0.00      0.00         0
          12       0.00      0.00      0.00         0
          13       0.00      0.00      0.00         0
          14       0.00      0.00      0.00         0

    accuracy                           0.58      1000
   macro avg       0.07      0.04      0.05      1000
weighted avg       1.00   

c:\Users\bugae\nlp\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bugae\nlp\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\bugae\nlp\venv\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## 3. GRU

In [148]:
import torch.nn as nn


class GRUModel(nn.Module):
    def __init__(self, input_size: int, hidden_size: int) -> None:
        super(GRUModel, self).__init__()
        self.gru = nn.GRU(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.linear = nn.Linear(hidden_size, 15)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        out, h = self.gru(x)
        y = self.linear(h.squeeze(0))
        return y


model_gru = GRUModel(INPUT_SIZE, HIDDEN_SIZE)
loss_gru = nn.CrossEntropyLoss()
optimizer_gru = torch.optim.Adam(model_gru.parameters(), lr=0.0025)

In [149]:
epochs = 20
for epoch in range(epochs):
    for x_b, y_b in train_dl:
        output = model_gru(x_b)
        loss_value = loss_gru(output, y_b.squeeze(1))
        optimizer_gru.zero_grad()
        loss_value.backward()
        optimizer_gru.step()
        
    print(f'Эпоха {epoch + 1}, Значение функции потерь: {loss_value.item()}')

Эпоха 1, Значение функции потерь: 1.4050920009613037
Эпоха 2, Значение функции потерь: 1.361263632774353
Эпоха 3, Значение функции потерь: 0.8475349545478821
Эпоха 4, Значение функции потерь: 0.5927150845527649
Эпоха 5, Значение функции потерь: 0.46416011452674866
Эпоха 6, Значение функции потерь: 0.3806973993778229
Эпоха 7, Значение функции потерь: 0.3199363946914673
Эпоха 8, Значение функции потерь: 0.2932031452655792
Эпоха 9, Значение функции потерь: 0.29936304688453674
Эпоха 10, Значение функции потерь: 0.22186987102031708
Эпоха 11, Значение функции потерь: 0.21176975965499878
Эпоха 12, Значение функции потерь: 0.17546552419662476
Эпоха 13, Значение функции потерь: 0.15659837424755096
Эпоха 14, Значение функции потерь: 0.12313138693571091
Эпоха 15, Значение функции потерь: 0.11684023588895798
Эпоха 16, Значение функции потерь: 0.1200065016746521
Эпоха 17, Значение функции потерь: 0.0896482914686203
Эпоха 18, Значение функции потерь: 0.09050662815570831
Эпоха 19, Значение функции по

In [150]:
y_pred_gru = np.argmax(sm(model_gru(torch.from_numpy(X_test).type(torch.float32))).detach().numpy(), axis=1)
print(classification_report(y_pred_gru, y_test))

              precision    recall  f1-score   support

           0       0.94      0.95      0.94       574
           1       0.80      1.00      0.89         4
           2       0.93      0.81      0.87        16
           3       0.69      0.69      0.69        13
           4       1.00      1.00      1.00         4
           5       0.95      0.95      0.95        37
           6       0.94      0.97      0.95        30
           7       0.50      1.00      0.67         2
           8       0.85      0.85      0.85        20
           9       0.25      0.50      0.33         2
          10       0.71      0.56      0.63         9
          11       0.92      0.87      0.89       248
          12       0.82      0.69      0.75        13
          13       0.73      0.73      0.73        26
          14       0.50      1.00      0.67         2

    accuracy                           0.91      1000
   macro avg       0.77      0.84      0.79      1000
weighted avg       0.91   